In [1]:
import numpy as np
from collections import defaultdict
from qiskit import Aer
from qiskit.visualization import plot_histogram
from qiskit.utils import QuantumInstance
from qiskit.algorithms import Grover, AmplificationProblem
from qiskit.circuit.library import PhaseOracle
#from qiskit.test.mock import FakeQuito
from qiskit.compiler import transpile
from qiskit import assemble
from qiskit import IBMQ
from qiskit import execute
from qiskit.tools import job_monitor
import matplotlib.pyplot as plt
IBMQ.save_account('7dc4818e71875543fa83807f90f5c16ba8e2bd1e900fd1194592de5ebf1f5e48d85fe6299cdf8ec1e2eb079ee7bea62e9aad442108c56150ad594f84de3c59a8', overwrite=True)
IBMQ.load_account()
#provider = IBMQ.get_provider(hub = 'ibm-q')
provider = IBMQ.get_provider(hub='ibm-q-ncsu', group='nc-state', project='vlsi-cad-tools')
device = provider.get_backend('ibmq_kolkata')
#quito = FakeQuito()
circuit_list = []
verification_list = []

In [2]:
class Verifier():
    """Create an object that can be used to check whether
    an assignment satisfies a DIMACS file.
        Args:
            dimacs_file (str): path to the DIMACS file
    """
    def __init__(self, dimacs_file):
        with open(dimacs_file, 'r') as f:
            self.dimacs = f.read()

    def is_correct(self, guess):
        """Verifies a SAT solution against this object's
        DIMACS file.
            Args:
                guess (str): Assignment to be verified.
                             Must be string of 1s and 0s.
            Returns:
                bool: True if `guess` satisfies the
                           problem. False otherwise.
        """
        # Convert characters to bools & reverse
        guess = [bool(int(x)) for x in guess][::-1]
        for line in self.dimacs.split('\n'):
            line = line.strip(' 0')
            clause_eval = False
            for literal in line.split(' '):
                if literal in ['p', 'c']:
                    # line is not a clause
                    clause_eval = True
                    break
                if '-' in literal:
                    literal = literal.strip('-')
                    lit_eval = not guess[int(literal)-1]
                else:
                    lit_eval = guess[int(literal)-1]
                clause_eval |= lit_eval
            if clause_eval is False:
                return False
        return True

In [4]:
opt_lvl = 3
circuit_list.clear()
verification_list.clear()
N=9
for andIter in range(N*2,N*2+1):#the ands
    for orIter in range(andIter*2,(N-1)*(andIter+1)+1, N): #the ors
        for notIter in range(3, 9, 2): #not composition
            thenotcomp = notIter*0.1
            filename = "kol_dimacs/demosatN"+str(N)+"Ands"+str(andIter)+"Ors"+str(orIter)+"NotComp"+format(thenotcomp, '.1f')+".dimacs"
            with open(filename, 'r') as f:
                dimacs = f.read()
            #print(dimacs)
            oracle = PhaseOracle.from_dimacs_file(filename)
            oracle.draw()
            v = Verifier(filename) 
            verification_list.append(v)
            #v.is_correct('101')
            # Configure backend
            backend = Aer.get_backend('aer_simulator')
            quantum_instance = QuantumInstance(backend, shots=1024)

            # Create a new problem from the phase oracle and the
            # verification function
            problem = AmplificationProblem(oracle=oracle, is_good_state=v.is_correct)

            # Use Grover's algorithm to solve the problem
            grover = Grover(quantum_instance=quantum_instance)
            result = grover.amplify(problem)
            '''
            result.top_measurement


            dd = defaultdict(list)

            for d in result.circuit_results : # you can list as many input dicts as you want here
                for key, value in d.items():
                    dd[key].append(value)
            for key in dd:
                dd[key] = sum(dd[key])
            print(dd)
            '''

            #print(result.circuit_results)
            #plot_histogram(result.circuit_results[-1])

            # transpile the circuit for ibmq_belem
            #print(max(result.iterations))
            qc = grover.construct_circuit(problem, max(result.iterations))
            qc.measure_all()
            #grover_compiled = transpile(qc, backend=quito, optimization_level=opt_lvl)

            #print('gates = ', grover_compiled.count_ops())
            #print('depth = ', grover_compiled.depth())

            #counts = quito.run(grover_compiled).result().get_counts()
            #plot_histogram(counts)

            qc_transpiled = transpile(qc, optimization_level=opt_lvl)
            print(qc_transpiled.depth())
            #qc_transpiled.draw(output='mpl')
            #plt.show()
            #print(qc_transpiled.decompose())
            circuit_list.append(qc_transpiled)

3115
8392
7422
11775
12233
25419
15809
18500
26299
16418
28788
19606
44367
32334
62371
44593
64685
44592
40521
57175
50757
96059
79460
71391
61638
73922
76570
60691
75325
80646
116021
106834
78003
107037
121517
118868
111453
107891
153207


In [6]:
##import pickle
##open_file = open("kol_dimacs/comp_list.pkl", "wb")
##open_verify_file = open("kol_dimacs/veri_list.pkl", "wb")
##pickle.dump(circuit_list, open_file)
##pickle.dump(verification_list, open_verify_file)

##open_file.close()
##open_verify_file.close()

In [4]:
import pickle
open_file = open("kol_dimacs/comp_list.pkl", "rb")
circuit_list = pickle.load(open_file)
open_file.close()

open_verify_file = open("kol_dimacs/veri_list.pkl", "rb")
verification_list = pickle.load(open_verify_file)
open_file.close()

AttributeError: Can't get attribute 'CircuitInstruction' on <module 'qiskit.circuit.quantumcircuitdata' from '/Users/ahmadbennakhi/opt/anaconda3/envs/QC/lib/python3.9/site-packages/qiskit/circuit/quantumcircuitdata.py'>

In [ ]:
#job = device.retrieve_job("6321d827e150f112839ddd0c")
#print(job)

In [8]:
job = execute(circuit_list,backend = device,shots = 8192)
print(job.job_id())

IBMQBackendApiError: 'Error submitting job: "(\'Connection aborted.\', ConnectionResetError(54, \'Connection reset by peer\'))"'

In [ ]:
job_monitor(job)
device_result = job.result()
#plot_histogram(device_result.get_counts())

In [ ]:
#v = Verifier('dimacs/demosatN3Ands7Ors16NotComp0.7.dimacs')
#v.is_correct('111')
job_counter = 0
N=4
for andIter in range(10,12):#the ands
    for orIter in range(andIter*2,(N-1)*(andIter+1)+1): #the ors
        for notIter in range(3, 9, 2): #not composition
            thenotcomp = notIter*0.1
            filename = "kol_dimacs/demosatN"+str(N)+"Ands"+str(andIter)+"Ors"+str(orIter)+"NotComp"+format(thenotcomp, '.1f')+".dimacs"
            v = Verifier(filename)
            #print(max(device_result.get_counts()[job_counter], key=device_result.get_counts()[job_counter].get))
            print(v.is_correct(max(device_result.get_counts()[job_counter], key=device_result.get_counts()[job_counter].get)))
            job_counter +=1

In [ ]:
print(len(device_result.get_counts()))